## Imports

In [75]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

## Dataframe Loading and Song Selection

In [76]:
# Load the dataframe
df = pd.read_csv('spotify_songs.csv')

valence  year  acousticness  \
0   0.0594  1921         0.982   
1   0.9630  1921         0.732   
2   0.0394  1921         0.961   
3   0.1650  1921         0.967   
4   0.2530  1921         0.957   

                                             artists  danceability  \
0  ['Sergei Rachmaninoff', 'James Levine', 'Berli...         0.279   
1                                     ['Dennis Day']         0.819   
2  ['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...         0.328   
3                                   ['Frank Parker']         0.275   
4                                     ['Phil Regan']         0.418   

   duration_ms  energy  explicit                      id  instrumentalness  \
0       831667   0.211         0  4BJqT0PrAfrxzMOxytFOIz          0.878000   
1       180533   0.341         0  7xPhfUan2yNtyFG0cUWkt8          0.000000   
2       500062   0.166         0  1o6I8BglA6ylDMrIELygv1          0.913000   
3       210000   0.309         0  3ftBPsC5vPBKxYSee08FDH          0.000028   
4       166693   0.193         0  4d6HGyGT8e121BsdKmw9v6          0.000002   

   key  liveness  loudness  mode  \
0   10     0.665   -20.096     1   
1    7     0.160   -12.441     1   
2    3     0.101   -14.850     1   
3    5     0.381    -9.316     1   
4    3     0.229   -10.096     1   

                                                name  popularity release_date  \
0  Piano Concerto No. 3 in D Minor, Op. 30: III. ...           4         1921   
1                            Clancy Lowered the Boom           5         1921   
2                                          Gati Bali           5         1921   
3                                          Danny Boy           3         1921   
4                        When Irish Eyes Are Smiling           2         1921   

   speechiness    tempo  
0       0.0366   80.954  
1       0.4150   60.936  
2       0.0339  110.339  
3       0.0354  100.109  
4       0.0380  101.665

In [77]:
# Enter a song title
song_title = input("Enter a song title to find similar songs: ")

# Normalize the input for case and remove leading or trailing whitespace
target_song_title = song_title.lower().strip()

# Check if the target_song_title is in the dataset
while target_song_title not in df['name'].str.lower().str.strip().values:
    print("- - - - - ")
    print(f"Error: Song '{target_song_title}' not found in the dataset.")
    target_song_title = input("Enter a song title to find similar songs: ")
    target_song_title = target_song_title.lower().strip()

Enter a song title to find similar songs: 
- - - - - 
Error: Song '' not found in the dataset.
Enter a song title to find similar songs: danny boy


In [78]:
# Retrieve all artists for a particular song title and sort alphabetically
available_artists = df.loc[df['name'].str.lower().str.strip() == target_song_title, 'artists'].unique()
available_artists.sort(axis=0)
print("Available artists:")
for i, artist in enumerate(available_artists):
    print(f"{i}. {artist}")

# Prompt user for artist input
artist_choice = input("Select the artist (enter the corresponding number): ")

# Error handling
try:
    artist_choice = int(artist_choice)
except:
    pass

while type(artist_choice) != int or artist_choice < 0 or artist_choice >= len(available_artists):
    print("- - - - - ")
    print("Invalid choice. Please select a valid artist.")
    
    for i, artist in enumerate(available_artists):
        print(f"{i}. {artist}")
    
    artist_choice = input("Select the artist (enter the corresponding number): ")
    
    try:
        artist_choice = int(artist_choice)
    except:
        pass

# Assign artist choice to variable
target_artist = available_artists[artist_choice]

Available artists:
0. ['Andy Williams']
1. ['Ben Webster']
2. ['Bill Evans', 'Shelly Manne']
3. ['Bill Evans']
4. ['Celtic Woman']
5. ['Conway Twitty']
6. ['Frank Parker']
7. ['Jackie Wilson']
8. ['Johnny Cash']
9. ['Judy Garland']
10. ['Patti Page']
11. ['Ray Price']
12. ['Traditional', 'Mario Lanza']
Select the artist (enter the corresponding number): 8


In [79]:
# # Retrieve all years for an artists for a particular song title and sort alphabetically
available_years = df.loc[(df['name'].str.lower().str.strip() == target_song_title) & (df['artists'] == target_artist), 'year'].unique()
available_years.sort(axis=0)

print(f"Available song years for {target_artist}:")
for i, year in enumerate(available_years):
    print(f"{i}. {year}")

# Prompt user for song year
year_choice = input("Select the song year (enter the corresponding number): ")

# Error handling
try:
    year_choice = int(year_choice)
except:
    pass

while type(year_choice) != int or year_choice < 0 or year_choice >= len(available_years):
    print("- - - - - ")
    print(f"Invalid choice. Please select a valid song year for {target_artist}.")
    for i, year in enumerate(available_years):
        print(f"{i}. {year}")
    year_choice = input("Select the song year (enter the corresponding number): ")

    try:
        year_choice = int(year_choice)
    except:
        pass

# Assign year choice to variable
target_year = available_years[year_choice]

Available song years for ['Johnny Cash']:
0. 1965
1. 2002
Select the song year (enter the corresponding number): 1


In [80]:
# Retrieve the target song based on name, artist, and year
target_song = df[(df['name'].str.lower().str.strip() == target_song_title) & (df['artists'] == target_artist) & (df['year'] == target_year)]

In [81]:
target_song

valence  year  acousticness          artists  danceability  \
121411    0.241  2002         0.983  ['Johnny Cash']         0.334   

        duration_ms  energy  explicit                      id  \
121411       198507  0.0595         0  5o4SqGekEfvdkNuOVx5d3S   

        instrumentalness  key  liveness  loudness  mode       name  \
121411          0.000087    9     0.124   -11.069     1  Danny Boy   

        popularity release_date  speechiness   tempo  
121411          48   2002-01-01       0.0416  177.07

In [82]:
# Retrieve index for selected song
target_song_index = target_song.index
target_song_index

Int64Index([121411], dtype='int64')

## Preprocessing

In [83]:
# Index dataframe by song names and drop unnecessary columns
df = df.set_index('name')
df = df.drop(columns=['artists', 'id', 'release_date'])

# Preprocess all other features and create scaled dataframe
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)

df_scaled.head()

valence     year  \
name                                                                    
Piano Concerto No. 3 in D Minor, Op. 30: III. F... -1.782825 -2.15247   
Clancy Lowered the Boom                             1.650688 -2.15247   
Gati Bali                                          -1.858821 -2.15247   
Danny Boy                                          -1.381564 -2.15247   
When Irish Eyes Are Smiling                        -1.047180 -2.15247   

                                                    acousticness  \
name                                                               
Piano Concerto No. 3 in D Minor, Op. 30: III. F...      1.276187   
Clancy Lowered the Boom                                 0.611347   
Gati Bali                                               1.220340   
Danny Boy                                               1.236296   
When Irish Eyes Are Smiling                             1.209703   

                                                    danceability  duration_ms  \
name                                                                            
Piano Concerto No. 3 in D Minor, Op. 30: III. F...     -1.467013     4.763146   
Clancy Lowered the Boom                                 1.598779    -0.399747   
Gati Bali                                              -1.188820     2.133824   
Danny Boy                                              -1.489722    -0.166101   
When Irish Eyes Are Smiling                            -0.677855    -0.509485   

                                                      energy  explicit  \
name                                                                     
Piano Concerto No. 3 in D Minor, Op. 30: III. F... -1.013988 -0.303955   
Clancy Lowered the Boom                            -0.528270 -0.303955   
Gati Bali                                          -1.182122 -0.303955   
Danny Boy                                          -0.647832 -0.303955   
When Irish Eyes Are Smiling                        -1.081242 -0.303955   

                                                    instrumentalness  \
name                                                                   
Piano Concerto No. 3 in D Minor, Op. 30: III. F...          2.268102   
Clancy Lowered the Boom                                    -0.532771   
Gati Bali                                                   2.379754   
Danny Boy                                                  -0.532682   
When Irish Eyes Are Smiling                                -0.532765   

                                                         key  liveness  \
name                                                                     
Piano Concerto No. 3 in D Minor, Op. 30: III. F...  1.365588  2.626719   
Clancy Lowered the Boom                             0.512123 -0.262229   
Gati Bali                                          -0.625830 -0.599749   
Danny Boy                                          -0.056853  1.002043   
When Irish Eyes Are Smiling                        -0.625830  0.132499   

                                                    loudness      mode  \
name                                                                     
Piano Concerto No. 3 in D Minor, Op. 30: III. F... -1.514237  0.643912   
Clancy Lowered the Boom                            -0.170766  0.643912   
Gati Bali                                          -0.593551  0.643912   
Danny Boy                                           0.377680  0.643912   
When Irish Eyes Are Smiling                         0.240788  0.643912   

                                                    popularity  speechiness  \
name                                                                          
Piano Concerto No. 3 in D Minor, Op. 30: III. F...   -1.256808    -0.379706   
Clancy Lowered the Boom                              -1.210993     1.945481   
Gati Bali                                            -1.210993    -0.396297   
Danny Boy                              

In [84]:
# Retrieve specific song features via index
target_song = df_scaled.iloc[target_song_index]
target_song

valence      year  acousticness  danceability  duration_ms  \
name                                                                     
Danny Boy -1.092778  0.972798      1.278846     -1.154756     -0.25723   

             energy  explicit  instrumentalness     key  liveness  loudness  \
name                                                                          
Danny Boy -1.580037 -0.303955         -0.532492  1.0811 -0.468173  0.070024   

               mode  popularity  speechiness     tempo  
name                                                    
Danny Boy  0.643912    0.759085    -0.348982  1.960647

## Calculate Euclidean Distances

In [85]:
# Calculate euclidean distances between the target song and all other songs
distances = euclidean_distances(target_song, df_scaled)

In [86]:
distances

array([[8.35101421, 7.09366253, 6.05740865, ..., 6.27223438, 7.57691089,
        6.78702779]])

In [87]:
len(distances)

1

In [88]:
# Insert calculated distances from target song into original dataframe at column position 0
df_distance_col = distances[0]
df.insert(0, 'distance', df_distance_col)
df

distance  valence  year  \
name                                                                          
Piano Concerto No. 3 in D Minor, Op. 30: III. F...  8.351014   0.0594  1921   
Clancy Lowered the Boom                             7.093663   0.9630  1921   
Gati Bali                                           6.057409   0.0394  1921   
Danny Boy                                           4.992615   0.1650  1921   
When Irish Eyes Are Smiling                         4.908085   0.2530  1921   
...                                                      ...      ...   ...   
China                                               5.661983   0.6080  2020   
Halloweenie III: Seven Days                         4.842598   0.7340  2020   
AYA                                                 6.272234   0.6370  2020   
Darkness                                            7.576911   0.1950  2020   
Billetes Azules (with J Balvin)                     6.787028   0.6420  2020   

                                                    acousticness  \
name                                                               
Piano Concerto No. 3 in D Minor, Op. 30: III. F...       0.98200   
Clancy Lowered the Boom                                  0.73200   
Gati Bali                                                0.96100   
Danny Boy                                                0.96700   
When Irish Eyes Are Smiling                              0.95700   
...                                                          ...   
China                                                    0.08460   
Halloweenie III: Seven Days                              0.20600   
AYA                                                      0.10100   
Darkness                                                 0.00998   
Billetes Azules (with J Balvin)                          0.13200   

                                                    danceability  duration_ms  \
name                                                                            
Piano Concerto No. 3 in D Minor, Op. 30: III. F...         0.279       831667   
Clancy Lowered the Boom                                    0.819       180533   
Gati Bali                                                  0.328       500062   
Danny Boy                                                  0.275       210000   
When Irish Eyes Are Smiling                                0.418       166693   
...                                                          ...          ...   
China                                                      0.786       301714   
Halloweenie III: Seven Days                                0.717       150654   
AYA                                                        0.634       211280   
Darkness                                                   0.671       337147   
Billetes Azules (with J Balvin)                            0.856       189507   

                                                    energy  explicit  \
name                                                                   
Piano Concerto No. 3 in D Minor, Op. 30: III. F...   0.211         0   
Clancy Lowered the Boom                              0.341         0   
Gati Bali                                            0.166         0   
Danny Boy                                            0.309         0   
When Irish Eyes Are Smiling                          0.193         0   
...                                                    ...       ...   
China                                                0.808         0   
Halloweenie III: Seven Days                          0.753         0   
AYA                                                  0.858         0   
Darkness                                             0.623         1   
Billetes Azules (with J Balvin)                      0.721         1   

                                                    instrumentalness  key  \
name                                                                   

## Sort Dataframe by Distance to Selected Song

In [89]:
# Sort the dataframe by distance
df_sorted = df.sort_values(by='distance')

df_sorted.head()

distance  valence  year  acousticness  danceability  \
name                                                                          
Danny Boy               0.000000    0.241  2002         0.983         0.334   
How Great Thou Art      0.793372    0.174  2006         0.853         0.348   
Gimme Gimme             0.901353    0.210  2002         0.886         0.328   
S'il suffisait d'aimer  0.975680    0.257  1998         0.921         0.346   
Mad About The Boy       0.996125    0.263  1987         0.922         0.344   

                        duration_ms  energy  explicit  instrumentalness  key  \
name                                                                           
Danny Boy                    198507  0.0595         0          0.000087    9   
How Great Thou Art           212827  0.2040         0          0.000002    8   
Gimme Gimme                  214227  0.2130         0          0.000000   11   
S'il suffisait d'aimer       214827  0.1970         0          0.000002   10   
Mad About The Boy            182573  0.1470         0          0.000190    8   

                        liveness  loudness  mode  popularity  speechiness  \
name                                                                        
Danny Boy                 0.1240   -11.069     1          48       0.0416   
How Great Thou Art        0.1390   -10.576     1          47       0.0292   
Gimme Gimme               0.1130   -10.698     1          43       0.0469   
S'il suffisait d'aimer    0.1340   -14.641     1          56       0.0393   
Mad About The Boy         0.0482   -10.300     1          58       0.0369   

                          tempo  
name                             
Danny Boy               177.070  
How Great Thou Art      174.519  
Gimme Gimme             176.136  
S'il suffisait d'aimer  176.646  
Mad About The Boy       177.192

In [90]:
df_sorted.describe()

distance        valence           year   acousticness  \
count  170653.000000  170653.000000  170653.000000  170653.000000   
mean        5.281924       0.528587    1976.787241       0.502115   
std         1.224310       0.263171      25.917853       0.376032   
min         0.000000       0.000000    1921.000000       0.000000   
25%         4.551535       0.317000    1956.000000       0.102000   
50%         5.230170       0.540000    1977.000000       0.516000   
75%         5.908968       0.747000    1999.000000       0.893000   
max        41.900335       1.000000    2020.000000       0.996000   

        danceability   duration_ms         energy       explicit  \
count  170653.000000  1.706530e+05  170653.000000  170653.000000   
mean        0.537396  2.309483e+05       0.482389       0.084575   
std         0.176138  1.261184e+05       0.267646       0.278249   
min         0.000000  5.108000e+03       0.000000       0.000000   
25%         0.415000  1.698270e+05       0.255000       0.000000   
50%         0.548000  2.074670e+05       0.471000       0.000000   
75%         0.668000  2.624000e+05       0.703000       0.000000   
max         0.988000  5.403500e+06       1.000000       1.000000   

       instrumentalness            key       liveness       loudness  \
count     170653.000000  170653.000000  170653.000000  170653.000000   
mean           0.167010       5.199844       0.205839     -11.467990   
std            0.313475       3.515094       0.174805       5.697943   
min            0.000000       0.000000       0.000000     -60.000000   
25%            0.000000       2.000000       0.098800     -14.615000   
50%            0.000216       5.000000       0.136000     -10.580000   
75%            0.102000       8.000000       0.261000      -7.183000   
max            1.000000      11.000000       1.000000       3.855000   

                mode     popularity    speechiness          tempo  
count  170653.000000  170653.000000  170653.000000  170653.000000  
mean        0.706902      31.431794       0.098393     116.861590  
std         0.455184      21.826615       0.162740      30.708533  
min         0.000000       0.000000       0.000000       0.000000  
25%         0.000000      11.000000       0.034900      93.421000  
50%         1.000000      33.000000       0.045000     114.729000  
75%         1.000000      48.000000       0.075600     135.537000  
max         1.000000     100.000000       0.970000     243.507000

In [91]:
# Display the top n most similar songs based on minimal distance to selected song
n = 5
df_sorted.head(n+1)

distance  valence  year  acousticness  danceability  \
name                                                                          
Danny Boy               0.000000    0.241  2002         0.983         0.334   
How Great Thou Art      0.793372    0.174  2006         0.853         0.348   
Gimme Gimme             0.901353    0.210  2002         0.886         0.328   
S'il suffisait d'aimer  0.975680    0.257  1998         0.921         0.346   
Mad About The Boy       0.996125    0.263  1987         0.922         0.344   
Baby Mine               1.018232    0.247  2007         0.906         0.362   

                        duration_ms  energy  explicit  instrumentalness  key  \
name                                                                           
Danny Boy                    198507  0.0595         0          0.000087    9   
How Great Thou Art           212827  0.2040         0          0.000002    8   
Gimme Gimme                  214227  0.2130         0          0.000000   11   
S'il suffisait d'aimer       214827  0.1970         0          0.000002   10   
Mad About The Boy            182573  0.1470         0          0.000190    8   
Baby Mine                    217360  0.2420         0          0.000009   11   

                        liveness  loudness  mode  popularity  speechiness  \
name                                                                        
Danny Boy                 0.1240   -11.069     1          48       0.0416   
How Great Thou Art        0.1390   -10.576     1          47       0.0292   
Gimme Gimme               0.1130   -10.698     1          43       0.0469   
S'il suffisait d'aimer    0.1340   -14.641     1          56       0.0393   
Mad About The Boy         0.0482   -10.300     1          58       0.0369   
Baby Mine                 0.0980   -11.356     1          52       0.0299   

                          tempo  
name                             
Danny Boy               177.070  
How Great Thou Art      174.519  
Gimme Gimme             176.136  
S'il suffisait d'aimer  176.646  
Mad About The Boy       177.192  
Baby Mine               184.428